In [1]:
import datetime

import pandas as pd

In [2]:
ratings = pd.read_csv(r'data/ratings.csv')
ratings.set_index(['userID', 'placeID'])

ratings.size

5805

In [3]:
restaurants = pd.read_csv(r'data/restaurants.csv')
restaurants.set_index('placeID')

print(restaurants.size)
restaurants.state.value_counts()

2730


SLP                50
Morelos            19
?                  18
San Luis Potosi    14
tamaulipas          9
Tamaulipas          7
san luis potosi     4
mexico              2
S.L.P.              2
slp                 2
morelos             1
san luis potos      1
s.l.p.              1
Name: state, dtype: int64

In [4]:
users = pd.read_csv(r'data/users.csv')
users.set_index('userID')
current_year = datetime.datetime.now().year
users['age'] = current_year - users.birth_year

print(users.size)

2760


In [5]:
def is_in_slp_state(restaurant):
    normalized_state_name = restaurants.state.str.lower().str.replace('[^a-zA-Z]', '')
    return normalized_state_name.isin(['slp', 'sanluispotosi'])


target_restaurants = restaurants[is_in_slp_state(restaurants)]
target_restaurants.state.value_counts()

SLP                50
San Luis Potosi    14
san luis potosi     4
S.L.P.              2
slp                 2
s.l.p.              1
Name: state, dtype: int64

In [6]:
target_users = users[(30 <= users.age) & (users.age <= 40)]
target_users.age.value_counts()

30    30
32    10
31     9
37     3
36     3
34     3
33     3
35     2
40     1
38     1
Name: age, dtype: int64

In [7]:
target_ratings = ratings[ratings.userID.isin(target_users.userID) & ratings.placeID.isin(target_restaurants.placeID)]
print(target_ratings.size)

1505


In [8]:
name_by_placeID = target_restaurants[['placeID','name']].set_index('placeID')
print(name_by_placeID.size, type(name_by_placeID))
name_by_placeID.head()

73 <class 'pandas.core.frame.DataFrame'>


,name
placeID,
132825,puesto de tacos
135106,El Rinc�n de San Francisco
135040,Restaurant los Compadres
135082,la Estrella de Dimas
135070,Restaurante 75


In [9]:
rating_mean_by_placeID = target_ratings[['placeID','rating']].groupby('placeID').mean()
print(rating_mean_by_placeID.size, type(rating_mean_by_placeID))
rating_mean_by_placeID.head()

73 <class 'pandas.core.frame.DataFrame'>


,rating
placeID,
132572,1.125000
132723,1.833333
132754,2.000000
132755,2.000000
132825,1.384615


In [10]:
user_count_by_placeID = target_ratings.groupby('placeID')['userID'].nunique()
user_count_by_placeID = pd.DataFrame({'placeID': user_count_by_placeID.index, 
                                      'count': user_count_by_placeID.values}).set_index('placeID')
print(user_count_by_placeID.size, type(user_count_by_placeID))
user_count_by_placeID.head()

73 <class 'pandas.core.frame.DataFrame'>


,count
placeID,
132572,8
132723,6
132754,2
132755,4
132825,13


In [11]:
output_df = name_by_placeID.join(rating_mean_by_placeID).join(user_count_by_placeID)
output_df.rename(columns={'rating': 'mean'}, inplace=True)
output_df = output_df.sort_values(['mean', 'count', 'name'], ascending=[False, False, True])
output_df.head()

,name,mean,count
placeID,,,
132755,La Estrella de Dimas,2.0,4
135034,Michiko Restaurant Japones,2.0,3
132754,Cabana Huasteca,2.0,2
135074,Restaurante la Parroquia Potosina,2.0,2
132854,Sirlone,2.0,2


In [12]:
output_df.to_csv(r'data/rez.csv', index=False)

In [13]:
with open(r'data/rez.csv', 'r') as f1:
    with open('data/expected_output.csv','r') as f2:
        assert f1.readlines() == f2.readlines()
        print("Test is passed")

Test is passed
